### Ячейка 1: Импорты

In [1]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 1: Импорты и Базовая Настройка (Возвращен IPYWIDGETS_AVAILABLE)
# =============================================================================
print("--- Ячейка 1: Импорты и Базовая Настройка ---")

# --- Базовые библиотеки ---
import numpy as np
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import zipfile
import time
import warnings
import json
import random
import traceback
from pathlib import Path
from typing import Union, Tuple, List, Dict, Optional

# --- Аудио и Сигналы ---
import librosa
import librosa.display
# import scipy # scipy.signal больше не нужен, т.к. нет фильтра

# --- Визуализация и Интерактивность ---
try:
    # === ВОЗВРАЩАЕМ ПРОВЕРКУ IPYWIDGETS ===
    import ipywidgets as widgets
    from IPython.display import display, Audio, clear_output
    IPYWIDGETS_AVAILABLE = True
    # =====================================
except ImportError:
    IPYWIDGETS_AVAILABLE = False # Определяем как False, если импорт не удался
    print("Предупреждение: ipywidgets не найден. Интерактивные ячейки (11) будут пропущены или вызовут ошибку.")
    # Определяем заглушки, чтобы код ниже не падал, если display/Audio нужны где-то еще
    def display(*args, **kwargs): pass
    def Audio(*args, **kwargs): pass
    def clear_output(*args, **kwargs): pass

import matplotlib.pyplot as plt

# --- PyTorch ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR

# --- Метрики и Утилиты ---
import Levenshtein
from tqdm.notebook import tqdm
import itertools

# --- Логирование Экспериментов ---
try:
    import mlflow
    import mlflow.pytorch
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False
    print("Предупреждение: mlflow не найден. Логирование экспериментов будет отключено.")

# --- Настройка окружения и предупреждений ---
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print("\n--- Статус ключевых библиотек ---")
print(f"PyTorch Версия: {torch.__version__}")
print(f"LibROSA Версия: {librosa.__version__}")
print(f"NumPy Версия: {np.__version__}")
print(f"Pandas Версия: {pd.__version__}")
print(f"Levenshtein Версия: {Levenshtein.__version__}")
print(f"MLflow доступен: {MLFLOW_AVAILABLE}")
print(f"IPyWidgets доступен: {IPYWIDGETS_AVAILABLE}") # Добавили вывод статуса
print("-" * 50)
# =============================================================================

--- Ячейка 1: Импорты и Базовая Настройка ---

--- Статус ключевых библиотек ---
PyTorch Версия: 2.5.1+cu121
LibROSA Версия: 0.11.0
NumPy Версия: 2.0.2
Pandas Версия: 2.2.3
Levenshtein Версия: 0.27.1
MLflow доступен: True
IPyWidgets доступен: True
--------------------------------------------------


### Ячейка 2: Конфигурация и Параметры

In [ ]:
# =============================================================================
# Ячейка 2: Конфигурация и Параметры (Обновлена Базовая Модель до 3x512)
# =============================================================================
print("--- Ячейка 2: Конфигурация и Параметры (Обновлена Базовая Модель до 3x512) ---")

import json
from pathlib import Path
import random
import numpy as np
import torch

SEED = 42

# --- Пути ---
BASE_DIR = Path('.').resolve()
DATA_DIR = BASE_DIR
ZIP_PATH = DATA_DIR / 'morse_dataset.zip'
AUDIO_DIR_NAME = 'morse_dataset'
EXTRACTED_AUDIO_DIR = DATA_DIR / AUDIO_DIR_NAME/ AUDIO_DIR_NAME
TRAIN_CSV_PATH = DATA_DIR / 'train.csv'
TEST_CSV_PATH = DATA_DIR / 'test.csv'
SAMPLE_SUB_PATH = DATA_DIR / 'sample_submission.csv'
# Папка для финального теста Trimming
OUTPUT_DIR = BASE_DIR / 'output_final_trim_test'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Базовая директория: {BASE_DIR}")
print(f"Ожидаемая директория аудио: {EXTRACTED_AUDIO_DIR}")
print(f"Директория для вывода: {OUTPUT_DIR}")

# --- Параметры обработки аудио (Готовы к тесту Trimming) ---
AUDIO_CONFIG = {
    "sample_rate": 8000,
    "frame_length_rms": 128,
    "hop_length_rms": 64,
    "apply_filter": False,
    "apply_trimming": False, # Будет меняться в Ячейке 13
    "trim_top_db": 0
}
print(f"\nАудио параметры (Готовы к тесту Trimming):")
print(json.dumps(AUDIO_CONFIG, indent=2))

# === ИЗМЕНЕНИЕ ЗДЕСЬ: Устанавливаем ЛУЧШУЮ МОДЕЛЬ (3x512) как базовую ===
MODEL_CONFIG_BASE = {
    "input_feature_dim": 2,
    "cnn_out_channels": [64, 128, 128],
    "cnn_kernel_size": 9, "cnn_stride": 1, "cnn_padding": 'same',
    "cnn_pool_kernel": 2,
    "rnn_hidden_size": 512, # <--- ЛУЧШИЙ РЕЗУЛЬТАТ
    "rnn_num_layers": 3,    # <--- ЛУЧШИЙ РЕЗУЛЬТАТ
    "dropout_rate": 0.2,    # Оставляем 0.2, т.к. 0.3 было хуже
    "activation_fn": "GELU",
    "classifier_type": "single" # Оставляем single, т.к. double было хуже
}
print(f"\nПараметры модели (ЛУЧШАЯ НАЙДЕННАЯ - 3x512):")
print(json.dumps(MODEL_CONFIG_BASE, indent=2))
# ============================================================================

# --- Параметры Обучения (Для ФИНАЛЬНОГО СРАВНЕНИЯ - можно оставить короткие эпохи) ---
# Оставим короткие эпохи для быстрого сравнения Trimming ON vs OFF
TRAIN_CONFIG = {
    "batch_size": 8, # Оставим 8, т.к. модель стала больше
    "num_workers": 0,
    "num_epochs": 15,
    "learning_rate": 2e-4,
    "div_factor": 5.0,
    "final_div_factor": 100,
    "weight_decay": 1e-4,
    "optimizer": "AdamW",
    "early_stopping_patience": 7,
    "gradient_clip_norm": 2.0,
    "validation_split_ratio": 0.1,
    "base_seed": SEED,
    "batches_per_epoch": 0 # <-- Короткие эпохи для сравнения

}
print(f"\nПараметры обучения (Для теста Trimming - короткие эпохи):")
print(json.dumps(TRAIN_CONFIG, indent=2))

# --- Специальные Токены и Глобальные Переменные ---
PAD_TOKEN = '<pad>'; BLANK_TOKEN = '_'
PAD_IDX = -1; BLANK_IDX = -1; NUM_CLASSES_CTC = -1
char_to_index: Dict[str, int] = {}; index_to_char: Dict[int, str] = {}

# --- Формирование Базового Имени Файла для финального теста ---
# Описывает лучшую модель (3x512)
BASE_FILENAME_SUFFIX_FINAL = (
    f"SR{AUDIO_CONFIG['sample_rate'] // 1000}k_"
    f"F{AUDIO_CONFIG['frame_length_rms']}h{AUDIO_CONFIG['hop_length_rms']}_FiltOFF_"
    f"CNN{MODEL_CONFIG_BASE['cnn_out_channels'][-1]}_"
    f"RNN{MODEL_CONFIG_BASE['rnn_num_layers']}x{MODEL_CONFIG_BASE['rnn_hidden_size']}_" # Лучшая 3x512
    f"{MODEL_CONFIG_BASE['activation_fn']}_Cls{MODEL_CONFIG_BASE['classifier_type']}_"
    f"LR{TRAIN_CONFIG['learning_rate']:.0e}_WD{TRAIN_CONFIG['weight_decay']:.0e}"
)
print(f"\nБазовый суффикс имени файла (Финал. тест): {BASE_FILENAME_SUFFIX_FINAL}")

# --- Выбор устройства ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nИспользуемое устройство: {device}")
if device.type == 'cuda': print(f"  GPU: {torch.cuda.get_device_name(0)}"); torch.cuda.empty_cache()

# --- Установка SEED ---
def set_seed(seed_value: int):
    random.seed(seed_value); np.random.seed(seed_value); torch.manual_seed(seed_value)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed_value)
    print(f"Установлен SEED = {seed_value}")
set_seed(TRAIN_CONFIG['base_seed'])

print("\n--- Ячейка 2: Конфигурация для ФИНАЛЬНОГО ТЕСТА Trimming готова ---")
print("-" * 50)
# =============================================================================

--- Ячейка 2: Конфигурация и Параметры (Обновлена Базовая Модель до 3x512) ---
Базовая директория: C:\Users\vasja\OneDrive\Рабочий стол\MorseAudioDecoder
Ожидаемая директория аудио: C:\Users\vasja\OneDrive\Рабочий стол\MorseAudioDecoder\morse_dataset\morse_dataset
Директория для вывода: C:\Users\vasja\OneDrive\Рабочий стол\MorseAudioDecoder\output_final_trim_test

Аудио параметры (Готовы к тесту Trimming):
{
  "sample_rate": 8000,
  "frame_length_rms": 128,
  "hop_length_rms": 64,
  "apply_filter": false,
  "apply_trimming": false,
  "trim_top_db": 30
}

Параметры модели (ЛУЧШАЯ НАЙДЕННАЯ - 3x512):
{
  "input_feature_dim": 2,
  "cnn_out_channels": [
    64,
    128,
    128
  ],
  "cnn_kernel_size": 9,
  "cnn_stride": 1,
  "cnn_padding": "same",
  "cnn_pool_kernel": 2,
  "rnn_hidden_size": 512,
  "rnn_num_layers": 3,
  "dropout_rate": 0.2,
  "activation_fn": "GELU",
  "classifier_type": "single"
}

Параметры обучения (Для теста Trimming - короткие эпохи):
{
  "batch_size": 8,
  "num_wo

### Ячейка 3: Загрузка данных и распаковка

In [3]:
# =============================================================================
# Ячейка 3: Загрузка Метаданных и Распаковка Аудио
# =============================================================================
print("--- Ячейка 3: Загрузка метаданных и распаковка аудио ---")

# --- Проверка и Распаковка Архива ---
if not EXTRACTED_AUDIO_DIR.exists():
    print(f"Папка для аудио ({EXTRACTED_AUDIO_DIR}) не найдена.")
    if ZIP_PATH.is_file():
        print(f"Найден архив: {ZIP_PATH}. Распаковка...")
        try:
            with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref: zip_ref.extractall(DATA_DIR)
            print(f"Архив распакован в: {DATA_DIR}")
            if not EXTRACTED_AUDIO_DIR.is_dir(): raise FileNotFoundError(f"Не удалось найти папку '{EXTRACTED_AUDIO_DIR.name}' после распаковки в {DATA_DIR}.")
            else: print(f"Папка с аудио найдена: {EXTRACTED_AUDIO_DIR}")
        except Exception as e: print(f"Критическая ошибка при распаковке: {e}"); traceback.print_exc(); raise SystemExit("Остановка.")
    else: print(f"Критическая ошибка: Архив {ZIP_PATH} не найден."); raise SystemExit("Остановка.")
else: print(f"Папка с аудио ({EXTRACTED_AUDIO_DIR}) уже существует. Распаковка пропускается.")

# --- Загрузка CSV файлов метаданных ---
print("\nЗагрузка CSV файлов...")
try:
    train_df = pd.read_csv(TRAIN_CSV_PATH)
    print(f"  Train DataFrame загружен: {len(train_df)} записей, Колонки: {train_df.columns.tolist()}")
    test_df = pd.read_csv(TEST_CSV_PATH)
    print(f"  Test DataFrame загружен: {len(test_df)} записей, Колонки: {test_df.columns.tolist()}")
    try: sample_sub_df = pd.read_csv(SAMPLE_SUB_PATH); print(f"  Sample Submission загружен: {len(sample_sub_df)} записей.")
    except FileNotFoundError: print(f"  Предупреждение: Файл Sample Submission ({SAMPLE_SUB_PATH}) не найден."); sample_sub_df = None
except FileNotFoundError as e: print(f"Критическая ошибка: Не найден CSV файл: {e}."); raise SystemExit("Остановка.")
except Exception as e: print(f"Критическая ошибка при чтении CSV: {e}"); traceback.print_exc(); raise SystemExit("Остановка.")

# --- Проверка наличия файлов и колонок ---
print("\nПроверка данных...")
if 'id' not in train_df.columns or 'message' not in train_df.columns: raise ValueError("В train_df нет 'id' или 'message'.")
if 'id' not in test_df.columns: raise ValueError("В test_df нет 'id'.")

# --- Выборочная проверка существования аудиофайлов ---
print("\nВыборочная проверка наличия аудиофайлов...")
if EXTRACTED_AUDIO_DIR.is_dir():
    num_check = 5; example_ids = train_df['id'].sample(min(num_check, len(train_df)), random_state=SEED).tolist(); missing_files = []
    for file_id in example_ids:
        if not (EXTRACTED_AUDIO_DIR / file_id).is_file(): missing_files.append(file_id)
    if not missing_files: print(f"  Проверено {len(example_ids)} файлов - все найдены (например, {EXTRACTED_AUDIO_DIR / example_ids[0]}).")
    else: print(f"  !!! ПРЕДУПРЕЖДЕНИЕ: Не найдены файлы для ID: {missing_files} !!!")
else: print(f"  Проверка невозможна: Папка {EXTRACTED_AUDIO_DIR} не существует.")

print("\n--- Ячейка 3: Загрузка данных завершена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 3: Загрузка метаданных и распаковка аудио ---
Папка с аудио (C:\Users\vasja\OneDrive\Рабочий стол\MorseAudioDecoder\morse_dataset\morse_dataset) уже существует. Распаковка пропускается.

Загрузка CSV файлов...
  Train DataFrame загружен: 30000 записей, Колонки: ['id', 'message']
  Test DataFrame загружен: 5000 записей, Колонки: ['id']
  Sample Submission загружен: 5000 записей.

Проверка данных...

Выборочная проверка наличия аудиофайлов...
  Проверено 5 файлов - все найдены (например, C:\Users\vasja\OneDrive\Рабочий стол\MorseAudioDecoder\morse_dataset\morse_dataset\2309.opus).

--- Ячейка 3: Загрузка данных завершена ---
--------------------------------------------------


### Ячейка 4: Создание словаря символов

In [4]:
# =============================================================================
# Ячейка 4: Создание Словаря Символов
# =============================================================================
print("--- Ячейка 4: Создание словаря символов ---")

if 'train_df' not in globals() or train_df is None: raise SystemExit("Остановка: train_df не найден.")
if 'message' not in train_df.columns: raise SystemExit("Остановка: В train_df нет 'message'.")

try:
    all_texts = train_df['message'].fillna('').astype(str)
    unique_chars = sorted(list(set(char for text in all_texts for char in text)))
    char_to_index = {char: i for i, char in enumerate(unique_chars)}
    index_to_char = {i: char for char, i in char_to_index.items()}
    BLANK_IDX = len(char_to_index)
    PAD_IDX = len(char_to_index) + 1
    char_to_index[BLANK_TOKEN] = BLANK_IDX; char_to_index[PAD_TOKEN] = PAD_IDX
    index_to_char[BLANK_IDX] = BLANK_TOKEN; index_to_char[PAD_IDX] = PAD_TOKEN
    NUM_CLASSES_CTC = BLANK_IDX + 1
    print(f"Найдено уникальных символов: {len(unique_chars)}")
    print(f"Размер словаря (включая BLANK и PAD): {len(char_to_index)}")
    print(f"Индекс BLANK ('{BLANK_TOKEN}'): {BLANK_IDX}, Индекс PAD ('{PAD_TOKEN}'): {PAD_IDX}")
    print(f"Количество классов для CTC Loss: {NUM_CLASSES_CTC}")
except Exception as e: print(f"Критическая ошибка при создании словаря: {e}"); traceback.print_exc(); raise SystemExit("Остановка.")
if not char_to_index or not index_to_char or BLANK_IDX == -1 or PAD_IDX == -1 or NUM_CLASSES_CTC <= 0: raise SystemExit("Остановка: Ошибка инициализации словаря.")

print("\n--- Ячейка 4: Создание словаря завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 4: Создание словаря символов ---
Найдено уникальных символов: 44
Размер словаря (включая BLANK и PAD): 46
Индекс BLANK ('_'): 44, Индекс PAD ('<pad>'): 45
Количество классов для CTC Loss: 45

--- Ячейка 4: Создание словаря завершено ---
--------------------------------------------------


### Ячейка 5: Класс MorseDataset

In [5]:
# =============================================================================
# Ячейка 5: Класс MorseDataset (Добавлена Обрезка Тишины)
# =============================================================================
print("--- Ячейка 5: Определение класса MorseDataset (Добавлена Обрезка Тишины) ---")

if 'BLANK_IDX' not in globals() or 'PAD_IDX' not in globals(): raise ValueError("Индексы BLANK/PAD не инициализированы.")
if 'AUDIO_CONFIG' not in globals(): raise ValueError("AUDIO_CONFIG не определен.")
if 'MODEL_CONFIG_BASE' not in globals(): raise ValueError("MODEL_CONFIG_BASE не определен.")

import numpy as np
import torch
from torch.utils.data import Dataset
import librosa
import pandas as pd
from pathlib import Path
import traceback
from typing import Union, Tuple, Optional, Dict, List

class MorseDataset(Dataset):
    """
    Датасет для Морзе.
    Выполняет: Загрузка -> Ресэмплинг -> [Опционально Trimming] -> RMS -> Delta -> Z-Score Нормализация.
    """
    def __init__(self,
                 dataframe: pd.DataFrame,
                 audio_dir: Path,
                 char_to_index: Dict[str, int],
                 audio_config: Dict,
                 model_input_feature_dim: int,
                 is_train: bool = True):
        super().__init__()
        if not isinstance(dataframe, pd.DataFrame): raise TypeError("dataframe должен быть pandas DataFrame")
        if not isinstance(audio_dir, Path): raise TypeError("audio_dir должен быть pathlib.Path")
        if not isinstance(char_to_index, dict): raise TypeError("char_to_index должен быть словарем")
        if not isinstance(audio_config, dict): raise TypeError("audio_config должен быть словарем")

        self.dataframe = dataframe.reset_index(drop=True)
        self.audio_dir = audio_dir
        self.char_to_index = char_to_index
        self.is_train = is_train
        self.audio_config = audio_config
        self.expected_feature_dim = model_input_feature_dim

        # --- Извлечение параметров из audio_config ---
        try:
            self.sample_rate = int(self.audio_config['sample_rate'])
            self.frame_length_rms = int(self.audio_config['frame_length_rms'])
            self.hop_length_rms = int(self.audio_config['hop_length_rms'])
            # === НОВЫЕ ПАРАМЕТРЫ ===
            self.apply_trimming = bool(self.audio_config.get('apply_trimming', False))
            self.trim_top_db = float(self.audio_config.get('trim_top_db', 30))
            # =======================
        except KeyError as e: raise ValueError(f"Отсутствует ключ в audio_config: {e}")
        except (TypeError, ValueError) as e: raise ValueError(f"Ошибка типа/значения в audio_config: {e}")

        # --- Валидация параметров ---
        if self.sample_rate <= 0: raise ValueError("sample_rate > 0")
        if self.frame_length_rms <= 0: raise ValueError("frame_length_rms > 0")
        if self.hop_length_rms <= 0: raise ValueError("hop_length_rms > 0")
        if self.apply_trimming and self.trim_top_db <= 0: raise ValueError("trim_top_db должен быть > 0, если apply_trimming=True")

        # --- Информационное сообщение ---
        trim_status = f"Trimming ON (top_db={self.trim_top_db})" if self.apply_trimming else "Trimming OFF"
        print(f"MorseDataset: is_train={self.is_train}, SR={self.sample_rate}Hz, {trim_status}")
        print(f"  Признаки: RMS(f={self.frame_length_rms}, h={self.hop_length_rms}) + Delta(standard)")
        print(f"  Нормализация: Z-Score")

    def __len__(self) -> int:
        return len(self.dataframe)

    def _normalize_feature(self, feature_array: np.ndarray) -> np.ndarray:
        """ Стандартная Z-score нормализация. """
        if not isinstance(feature_array, np.ndarray) or feature_array.size == 0: return np.array([], dtype=np.float32)
        epsilon = 1e-8; mean = np.mean(feature_array); std = np.std(feature_array)
        if std < epsilon: return np.zeros_like(feature_array, dtype=np.float32)
        return ((feature_array - mean) / (std + epsilon)).astype(np.float32)

    def _calculate_features(self, waveform_np: np.ndarray, file_id_for_log: str = "N/A") -> Optional[torch.Tensor]:
        """ Вычисляет признаки: RMS -> Delta -> Z-Score -> Стек. """
        empty_tensor = torch.empty((self.expected_feature_dim, 0), dtype=torch.float32)
        if not isinstance(waveform_np, np.ndarray) or waveform_np.size == 0: return empty_tensor
        try:
            processed_waveform = waveform_np.astype(np.float32)

            # 1. Расчет RMS
            rms_envelope_raw = librosa.feature.rms(y=processed_waveform, frame_length=self.frame_length_rms, hop_length=self.hop_length_rms, center=True, pad_mode='reflect')[0]
            if rms_envelope_raw.size < 2: return empty_tensor

            # 2. Расчет Дельты
            delta_raw = np.diff(rms_envelope_raw, n=1, prepend=rms_envelope_raw[0])

            # 3. Нормализация Z-Score
            norm_rms = self._normalize_feature(rms_envelope_raw)
            norm_delta = self._normalize_feature(delta_raw)

            # 4. Сборка тензора
            if norm_rms.size == 0 or norm_delta.size == 0 or norm_rms.shape != norm_delta.shape:
                 print(f"WARNING ({file_id_for_log}): Проблема с формой/содержимым признаков после нормализации. RMS:{norm_rms.shape}, Delta:{norm_delta.shape}")
                 return empty_tensor
            features_np = np.vstack([norm_rms, norm_delta]).astype(np.float32)
            features_tensor = torch.from_numpy(features_np)
            if not torch.isfinite(features_tensor).all(): print(f"ERROR ({file_id_for_log}): NaN/Inf в финальном тензоре! Пропуск."); return None
            return features_tensor
        except Exception as e: print(f"CRITICAL ERROR в _calculate_features ({file_id_for_log}): {e}"); traceback.print_exc(limit=1); return None

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, torch.Tensor], Tuple[torch.Tensor, str], Tuple[None, Optional[str]]]:
        """ Загрузка, обработка и возврат примера. """
        if not (0 <= index < len(self.dataframe)): return None, (f"InvalidIndex_{index}" if not self.is_train else None)
        try: row = self.dataframe.iloc[index]; file_id = row['id']; audio_path = self.audio_dir / file_id
        except Exception as e: print(f"Error get item data index {index}: {e}"); return None, (f"DataAccessError_{index}" if not self.is_train else None)

        # 1. Загрузка и Ресэмплинг
        waveform_np: Optional[np.ndarray] = None
        original_len = 0
        try:
            waveform_np, _ = librosa.load(audio_path, sr=self.sample_rate, mono=True)
            if waveform_np is None or waveform_np.size == 0: print(f"Warning ({file_id}): Файл пуст."); return None, (file_id if not self.is_train else None)
            original_len = len(waveform_np)
        except FileNotFoundError: print(f"ERROR ({file_id}): Файл не найден {audio_path}"); return None, (file_id if not self.is_train else None)
        except Exception as e: print(f"ERROR ({file_id}): Ошибка загрузки librosa: {e}"); traceback.print_exc(limit=1); return None, (file_id if not self.is_train else None)

        # === НОВЫЙ ШАГ: Обрезка тишины (Trimming) ===
        trimmed_len = original_len
        if self.apply_trimming:
            try:
                waveform_trimmed, _ = librosa.effects.trim(waveform_np, top_db=self.trim_top_db, frame_length=512, hop_length=128) # Параметры trim можно тоже тюнить
                if waveform_trimmed is not None and waveform_trimmed.size > 0:
                    waveform_np = waveform_trimmed
                    trimmed_len = len(waveform_np)
                # else: # Оставляем исходный, если trim вернул пустоту
                #     print(f"Warning ({file_id}): Trimming удалил весь сигнал, используется исходный.")
            except Exception as e_trim:
                print(f"ERROR ({file_id}): Ошибка librosa.effects.trim: {e_trim}. Используется исходный сигнал.")
        # ============================================

        # 2. Вычисление признаков (RMS -> Delta -> Z-Score)
        features: Optional[torch.Tensor] = self._calculate_features(waveform_np, file_id)
        if features is None or features.shape[1] == 0:
            # print(f"Debug ({file_id}): Features None/Empty. Orig len={original_len}, Trimmed len={trimmed_len}")
            return None, (file_id if not self.is_train else None)

        # 3. Подготовка цели или возврат ID
        if self.is_train:
            message_text = str(row.get('message', ''))
            target_indices = [self.char_to_index.get(c) for c in message_text if c in self.char_to_index]
            if not target_indices: return None, None
            target_tensor = torch.tensor(target_indices, dtype=torch.long)
            return features, target_tensor
        else: return features, file_id

print("\n--- Ячейка 5: Определение MorseDataset (Добавлена Обрезка Тишины) завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 5: Определение класса MorseDataset (Добавлена Обрезка Тишины) ---

--- Ячейка 5: Определение MorseDataset (Добавлена Обрезка Тишины) завершено ---
--------------------------------------------------


### Ячейка 6: Функция collate_fn

In [6]:
# =============================================================================
# Ячейка 6: Функция collate_fn (Сборка Батчей)
# =============================================================================
print("--- Ячейка 6: Определение функции collate_fn ---")

if 'PAD_IDX' not in globals(): raise ValueError("PAD_IDX не инициализирован!")
if 'torch' not in globals(): import torch
if 'pad_sequence' not in globals(): from torch.nn.utils.rnn import pad_sequence
from typing import List, Tuple, Optional, Union

def collate_fn(batch: List[Tuple[Optional[torch.Tensor], Optional[Union[torch.Tensor, str]]]]) \
    -> Optional[Tuple[torch.Tensor, Union[torch.Tensor, List[str]], torch.Tensor, Optional[torch.Tensor]]]:
    """ Собирает батч, фильтрует некорректные, выполняет паддинг. """
    valid_batch = [item for item in batch if item[0] is not None and item[1] is not None and item[0].shape[1] > 0]
    if not valid_batch: return None

    is_train_or_val_batch = isinstance(valid_batch[0][1], torch.Tensor)
    features_list = [item[0].permute(1, 0) for item in valid_batch] # (T, F)
    targets_or_ids_list = [item[1] for item in valid_batch]

    features_padded_time_first = pad_sequence(features_list, batch_first=True, padding_value=0.0)
    features_padded = features_padded_time_first.permute(0, 2, 1) # (B, F, T_max)
    feature_lengths = torch.tensor([f.shape[0] for f in features_list], dtype=torch.long)

    if is_train_or_val_batch:
        targets_list: List[torch.Tensor] = targets_or_ids_list
        targets_padded: torch.Tensor = pad_sequence(targets_list, batch_first=True, padding_value=PAD_IDX)
        target_lengths: torch.Tensor = torch.tensor([len(t) for t in targets_list], dtype=torch.long)
        return features_padded, targets_padded, feature_lengths, target_lengths
    else:
        file_ids: List[str] = targets_or_ids_list
        return features_padded, file_ids, feature_lengths, None

print("Функция collate_fn определена.")
print("\n--- Ячейка 6: Определение collate_fn завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 6: Определение функции collate_fn ---
Функция collate_fn определена.

--- Ячейка 6: Определение collate_fn завершено ---
--------------------------------------------------


### Ячейка 7: Модель MorseRecognizer

In [7]:
# =============================================================================
# Ячейка 7: Модель MorseRecognizer (Исправлен finally)
# =============================================================================
print("--- Ячейка 7: Определение модели MorseRecognizer (Обновлена) ---")

if 'MODEL_CONFIG_BASE' not in globals(): raise ValueError("MODEL_CONFIG_BASE не определен!")
if 'NUM_CLASSES_CTC' not in globals() or NUM_CLASSES_CTC <= 0: raise ValueError("NUM_CLASSES_CTC не корректен!")
if 'torch' not in globals(): import torch
if 'nn' not in globals(): import torch.nn as nn
from typing import Union, Tuple, List, Dict, Optional # Добавим импорт типов сюда на всякий случай

class MorseRecognizer(nn.Module):
    """ Модель для распознавания Морзе (CNN + BiGRU). Обновлена для поддержки разного типа классификатора. """
    def __init__(self, num_classes_ctc: int, input_feature_dim: int,
                 cnn_out_channels: List[int], cnn_kernel_size: int, cnn_stride: int,
                 cnn_padding: Union[int, str], cnn_pool_kernel: int,
                 rnn_hidden_size: int, rnn_num_layers: int, dropout_rate: float,
                 activation_fn: str = "GELU", classifier_type: str = "single"): # Добавлен classifier_type
        super().__init__()
        self.input_feature_dim = input_feature_dim
        self._time_reduction_factor = 1.0
        cnn_layers = []; in_channels = input_feature_dim
        try: ActivationLayer = getattr(nn, activation_fn)
        except AttributeError: print(f"Warning: Activation '{activation_fn}' не найдена. Используется GELU."); ActivationLayer = nn.GELU

        # CNN Extractor
        for i, out_channels in enumerate(cnn_out_channels):
            layer = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, cnn_kernel_size, cnn_stride, padding=cnn_padding),
                nn.BatchNorm1d(out_channels), ActivationLayer(),
                nn.MaxPool1d(cnn_pool_kernel), nn.Dropout(dropout_rate) )
            cnn_layers.append(layer); in_channels = out_channels
            self._time_reduction_factor *= cnn_pool_kernel
        self.cnn_extractor = nn.Sequential(*cnn_layers)
        self.cnn_output_dim = in_channels

        # RNN
        self.rnn = nn.GRU( self.cnn_output_dim, rnn_hidden_size, rnn_num_layers,
                           batch_first=True, bidirectional=True, dropout=dropout_rate if rnn_num_layers > 1 else 0.0 )
        rnn_output_dim = rnn_hidden_size * 2

        # Classifier (Single or Double Linear)
        self.classifier_type = classifier_type
        if self.classifier_type == "double":
            intermediate_dim = rnn_output_dim
            self.classifier = nn.Sequential(
                nn.Linear(rnn_output_dim, intermediate_dim),
                ActivationLayer(),
                nn.Dropout(dropout_rate),
                nn.Linear(intermediate_dim, num_classes_ctc)
            )
            classifier_str = f"DoubleLinear({rnn_output_dim}->{intermediate_dim}->{num_classes_ctc})"
        elif self.classifier_type == "single":
            self.classifier = nn.Linear(rnn_output_dim, num_classes_ctc)
            classifier_str = f"SingleLinear({rnn_output_dim}->{num_classes_ctc})"
        else:
            raise ValueError(f"Неизвестный classifier_type: {self.classifier_type}. Допустимы 'single', 'double'.")

        print(f"Архитектура MorseRecognizer:")
        print(f"  CNN: {len(cnn_out_channels)} layers, OutChannels={cnn_out_channels}, Kernel={cnn_kernel_size}, Pool={cnn_pool_kernel}")
        print(f"       Output Dim={self.cnn_output_dim}, Time Reduction Factor={self._time_reduction_factor:.1f}x")
        print(f"  RNN: BiGRU, Layers={rnn_num_layers}, Hidden Size={rnn_hidden_size}")
        print(f"       Output Dim={rnn_output_dim}")
        print(f"  Activation: {activation_fn}")
        print(f"  Classifier: {classifier_str}")
        print(f"  Dropout: {dropout_rate}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """ Input: (B, F, T_in) -> CNN -> (B, C, T_red) -> Permute -> (B, T_red, C) -> RNN -> (B, T_red, H*2) -> Classifier -> (B, T_red, N_classes) -> Permute -> Output: (T_red, B, N_classes) """
        if x.shape[1] != self.input_feature_dim: raise ValueError(f"Input feature dim mismatch! Expected {self.input_feature_dim}, got {x.shape[1]}. Shape: {x.shape}")
        x = self.cnn_extractor(x)   # (B, C_cnn, T_reduced)
        x = x.permute(0, 2, 1)      # (B, T_reduced, C_cnn)
        x_rnn, _ = self.rnn(x)      # (B, T_reduced, Hidden*2)
        logits = self.classifier(x_rnn) # (B, T_reduced, NumClasses)
        logits = logits.permute(1, 0, 2) # (T_reduced, B, NumClasses) - для CTC
        return logits

    def get_time_reduction_factor(self) -> float: return self._time_reduction_factor

# --- Создание и Проверка Экземпляра БАЗОВОЙ Модели ---
model_created_successfully = False
model = None # Объявляем переменные до try
dummy_input = None
dummy_output = None
try:
    print("\nСоздание экземпляра БАЗОВОЙ модели для проверки...")
    model = MorseRecognizer(
        num_classes_ctc=NUM_CLASSES_CTC,
        **MODEL_CONFIG_BASE # Используем базовый конфиг для проверки
    ).to(device)
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Модель '{type(model).__name__}' (базовая) создана ({total_params:,} параметров) на {device}.")

    print("\nПроверка forward pass базовой модели..."); model.eval()
    dummy_batch_size = 4; dummy_time_steps = 1000
    dummy_input = torch.randn(dummy_batch_size, MODEL_CONFIG_BASE['input_feature_dim'], dummy_time_steps).to(device)
    with torch.no_grad(): dummy_output = model(dummy_input)
    print(f"  Вход: {dummy_input.shape}, Выход: {dummy_output.shape}")
    expected_time_dim = int(dummy_time_steps / model.get_time_reduction_factor())
    if abs(dummy_output.shape[0] - expected_time_dim) > 2: print(f"  ПРЕДУПРЕЖДЕНИЕ: Неожиданная длина выхода! Ожидалось ~{expected_time_dim}, получено {dummy_output.shape[0]}.")
    assert dummy_output.shape[1] == dummy_batch_size, "Batch size mismatch!"
    assert dummy_output.shape[2] == NUM_CLASSES_CTC, "Num classes mismatch!"
    print("  Размерности выхода (T, B, C) корректны."); model_created_successfully = True
except Exception as e: print(f"\n!!! КРИТИЧЕСКАЯ ОШИБКА при создании/проверке модели: {e} !!!"); traceback.print_exc()
finally:
    # === ИСПРАВЛЕНИЕ ЗДЕСЬ ===
    # Каждая команда на новой строке с отступом
    if model is not None: del model
    if dummy_input is not None: del dummy_input
    if dummy_output is not None: del dummy_output
    if torch.cuda.is_available():
        torch.cuda.empty_cache() # Очищаем память после проверки
    # ========================

if not model_created_successfully: raise SystemExit("Остановка: Не удалось создать/проверить модель.")

print("\n--- Ячейка 7: Определение и проверка модели завершены (finally исправлен) ---")
print("-" * 50)
# =============================================================================

--- Ячейка 7: Определение модели MorseRecognizer (Обновлена) ---

Создание экземпляра БАЗОВОЙ модели для проверки...
Архитектура MorseRecognizer:
  CNN: 3 layers, OutChannels=[64, 128, 128], Kernel=9, Pool=2
       Output Dim=128, Time Reduction Factor=8.0x
  RNN: BiGRU, Layers=3, Hidden Size=512
       Output Dim=1024
  Activation: GELU
  Classifier: SingleLinear(1024->45)
  Dropout: 0.2
Модель 'MorseRecognizer' (базовая) создана (11,691,117 параметров) на cuda.

Проверка forward pass базовой модели...
  Вход: torch.Size([4, 2, 1000]), Выход: torch.Size([125, 4, 45])
  Размерности выхода (T, B, C) корректны.

--- Ячейка 7: Определение и проверка модели завершены (finally исправлен) ---
--------------------------------------------------


### Ячейка 8: Loss, Optimizer, Scheduler

In [8]:

print("--- Ячейка 8: Настройка Loss, Optimizer, Scheduler ---")

if 'BLANK_IDX' not in globals(): raise ValueError("BLANK_IDX не инициализирован!")
if 'TRAIN_CONFIG' not in globals(): raise ValueError("TRAIN_CONFIG не определен!")
if 'optim' not in globals(): import torch.optim as optim
if 'nn' not in globals(): import torch.nn as nn
if 'OneCycleLR' not in globals(): from torch.optim.lr_scheduler import OneCycleLR

criterion = nn.CTCLoss(blank=BLANK_IDX, reduction='mean', zero_infinity=True)
print(f"Функция потерь: CTCLoss (blank={BLANK_IDX}, reduction='mean', zero_infinity=True)")

optimizer_name = TRAIN_CONFIG.get('optimizer', 'AdamW').lower()
lr = TRAIN_CONFIG['learning_rate']; wd = TRAIN_CONFIG['weight_decay']
print(f"\nОптимизатор: {optimizer_name.upper()} (LR={lr:.1e}, WD={wd:.1e}) - будет создан в пайплайне")

div_f = TRAIN_CONFIG.get('div_factor', 25.0); final_div_f = TRAIN_CONFIG.get('final_div_factor', 1e4)
print(f"\nПланировщик: OneCycleLR (div={div_f}, final_div={final_div_f}) - будет создан в пайплайне")
print("\n--- Ячейка 8: Настройка Loss, Optimizer, Scheduler завершена ---")
print("-" * 50)

--- Ячейка 8: Настройка Loss, Optimizer, Scheduler ---
Функция потерь: CTCLoss (blank=44, reduction='mean', zero_infinity=True)

Оптимизатор: ADAMW (LR=2.0e-04, WD=1.0e-04) - будет создан в пайплайне

Планировщик: OneCycleLR (div=5.0, final_div=100) - будет создан в пайплайне

--- Ячейка 8: Настройка Loss, Optimizer, Scheduler завершена ---
--------------------------------------------------


### Ячейка 9: Функции Декодирования и Метрики

In [9]:


# =============================================================================
# Ячейка 9: Функции Декодирования (Greedy) и Метрики (Levenshtein)
# =============================================================================
print("--- Ячейка 9: Определение функций декодирования и метрики ---")

if 'index_to_char' not in globals(): raise ValueError("index_to_char не определен!")
if 'BLANK_IDX' not in globals(): raise ValueError("BLANK_IDX не определен!")
if 'PAD_IDX' not in globals(): raise ValueError("PAD_IDX не определен!")
if 'torch' not in globals(): import torch
if 'Levenshtein' not in globals(): import Levenshtein
from typing import List, Dict, Tuple

# --- Greedy CTC Decoding ---
def ctc_greedy_decode(logits: torch.Tensor, index_to_char_map: Dict[int, str], blank_idx: int) -> List[str]:
    """ Жадное CTC декодирование батча логитов (Time, Batch, Classes). """
    decoded_batch = []
    best_path = torch.argmax(logits, dim=2) # (Time, Batch)
    best_path_np = best_path.cpu().numpy()
    for i in range(best_path_np.shape[1]):
        sequence_indices = best_path_np[:, i]
        collapsed_indices = [idx for j, idx in enumerate(sequence_indices) if j == 0 or idx != sequence_indices[j-1]]
        final_indices = [idx for idx in collapsed_indices if idx != blank_idx]
        decoded_string = "".join([index_to_char_map.get(idx, '?') for idx in final_indices])
        decoded_batch.append(decoded_string)
    return decoded_batch

# --- Levenshtein Distance ---
def calculate_levenshtein(predictions: List[str], targets_padded: torch.Tensor, target_lengths: torch.Tensor,
                          index_to_char_map: Dict[int, str], pad_idx: int) -> Tuple[float, List[Tuple[str, str]]]:
    """ Вычисляет средний Levenshtein и возвращает пары (предсказание, реальность). """
    total_distance = 0.0; num_valid_pairs = 0; decoded_pairs = []
    targets_np = targets_padded.cpu().numpy(); target_lengths_np = target_lengths.cpu().numpy()
    batch_size = targets_padded.shape[0]
    if len(predictions) != batch_size: print(f"Warning: Levenshtein size mismatch! Preds:{len(predictions)}, Targets:{batch_size}"); return float('inf'), []
    for i in range(batch_size):
        real_target_len = target_lengths_np[i]; pred_str = predictions[i]
        if real_target_len <= 0: target_str = ""; dist = len(pred_str)
        else:
            target_indices = targets_np[i, :real_target_len]
            target_str = "".join([index_to_char_map.get(idx, '?') for idx in target_indices if idx != pad_idx])
            try: dist = Levenshtein.distance(pred_str, target_str)
            except Exception as e: print(f"Levenshtein Error: ('{pred_str}', '{target_str}'). {e}"); dist = max(len(pred_str), len(target_str))
        total_distance += dist; num_valid_pairs += 1; decoded_pairs.append((pred_str, target_str))
    mean_levenshtein = total_distance / num_valid_pairs if num_valid_pairs > 0 else float('inf')
    return mean_levenshtein, decoded_pairs

print("Функции ctc_greedy_decode и calculate_levenshtein определены.")
print("\n--- Ячейка 9: Определение функций декодирования и метрики завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 9: Определение функций декодирования и метрики ---
Функции ctc_greedy_decode и calculate_levenshtein определены.

--- Ячейка 9: Определение функций декодирования и метрики завершено ---
--------------------------------------------------


### Ячейка 10: Функции Обучения и Валидации Эпохи

In [10]:
# =============================================================================
# Ячейка 10: Функции Обучения и Валидации Эпохи (Исправлен SyntaxError в train_epoch)
# =============================================================================
print("--- Ячейка 10: Определение функций обучения и валидации эпохи ---")

# Зависимости проверяются в run_training_pipeline
if 'F' not in globals(): import torch.nn.functional as F
if 'tqdm' not in globals(): from tqdm.notebook import tqdm
if 'np' not in globals(): import numpy as np
if 'torch' not in globals(): import torch
if 'optim' not in globals(): import torch.optim as optim
if 'nn' not in globals(): import torch.nn as nn
if 'traceback' not in globals(): import traceback
if 'DataLoader' not in globals(): from torch.utils.data import DataLoader
if 'itertools' not in globals(): import itertools
from typing import Optional, Tuple, List, Dict

# --- Функция Обучения Одной Эпохи ---
def train_epoch(model: nn.Module, dataloader: DataLoader, criterion: nn.CTCLoss, optimizer: optim.Optimizer,
                scheduler: Optional[optim.lr_scheduler._LRScheduler], device: torch.device, epoch_num: int, total_epochs: int,
                index_to_char_map: Dict[int, str], blank_idx: int, pad_idx: int, grad_clip_norm: float,
                batches_per_epoch: int = 0) -> Tuple[float, float, float]:
    """ Выполняет одну эпоху обучения. """
    model.train(); running_loss = 0.0; total_lev_dist = 0.0; total_lr = 0.0; num_batches_processed = 0; total_samples = 0
    try: time_factor = max(model.get_time_reduction_factor(), 1.0)
    except AttributeError: time_factor = 1.0

    total_batches_in_loader = len(dataloader); iterator = dataloader; num_batches_to_process = total_batches_in_loader
    if batches_per_epoch > 0 and batches_per_epoch < total_batches_in_loader:
         iterator = itertools.islice(dataloader, batches_per_epoch); num_batches_to_process = batches_per_epoch
    if num_batches_to_process == 0: return 0.0, float('inf'), 0.0

    pbar = tqdm(iterator, total=num_batches_to_process, desc=f"Эпоха {epoch_num}/{total_epochs} [Тренировка]", leave=False, ncols=1000)
    for batch_idx, batch_data in enumerate(pbar):
        if batch_data is None: continue
        features, targets, feature_lengths, target_lengths = batch_data
        if features is None or targets is None: continue
        batch_size = features.size(0)
        if batch_size == 0: continue

        features = features.to(device, non_blocking=True); targets = targets.to(device, non_blocking=True)
        feature_lengths_cpu = feature_lengths.cpu(); target_lengths_cpu = target_lengths.cpu()
        input_lengths_ctc = torch.floor(feature_lengths_cpu.float() / time_factor + 1e-9).long().clamp(min=1)
        loss_value = float('inf'); lev_dist_batch = float('inf'); current_lr = optimizer.param_groups[0]['lr']
        try:
            optimizer.zero_grad()
            logits = model(features) # (T_red, B, C)
            if logits.shape[1] != batch_size: print(f"\n!!! ERROR (Train): Batch size mismatch! Out:{logits.shape[1]} != In:{batch_size}. Skip."); continue
            log_probs = F.log_softmax(logits, dim=2)
            output_length = log_probs.shape[0]
            input_lengths_ctc_clamped = input_lengths_ctc.clamp(max=output_length)
            target_lengths_clamped = target_lengths_cpu.clamp(max=targets.shape[1])
            valid_target_mask = target_lengths_clamped > 0
            if not torch.all(valid_target_mask):
                 log_probs_valid = log_probs[:, valid_target_mask, :]; targets_valid = targets[valid_target_mask, :]
                 input_lengths_valid = input_lengths_ctc_clamped[valid_target_mask]; target_lengths_valid = target_lengths_clamped[valid_target_mask]
                 loss = criterion(log_probs_valid, targets_valid, input_lengths_valid, target_lengths_valid) if log_probs_valid.shape[1] > 0 else torch.tensor(0.0, device=device)
            else: loss = criterion(log_probs, targets, input_lengths_ctc_clamped, target_lengths_clamped)
            if not torch.isfinite(loss): print(f"\n!!! WARNING (Train): NaN/Inf loss! Skip step."); optimizer.zero_grad(); continue
            loss_value = loss.item()
            if loss.requires_grad:
                loss.backward()
                if grad_clip_norm > 0: torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip_norm)
                optimizer.step()
                if scheduler: scheduler.step()
            with torch.no_grad():
                if 'ctc_greedy_decode' not in globals(): raise NameError("ctc_greedy_decode missing")
                decoded_preds = ctc_greedy_decode(logits, index_to_char_map, blank_idx)
                if 'calculate_levenshtein' not in globals(): raise NameError("calculate_levenshtein missing")
                lev_dist_batch, _ = calculate_levenshtein(decoded_preds, targets.cpu(), target_lengths_cpu, index_to_char_map, pad_idx)
                if not np.isfinite(lev_dist_batch): lev_dist_batch = 20.0
        except RuntimeError as e:
             if "CUDA out of memory" in str(e): print("\n!!! CUDA OOM (Train) !!!"); raise e
             else: print(f"\n!!! RuntimeError (Train): {e} !!!"); traceback.print_exc(limit=1); loss_value = 30.0; lev_dist_batch = 30.0
        except Exception as e: print(f"\n!!! Error (Train): {e} !!!"); traceback.print_exc(limit=1); loss_value = 30.0; lev_dist_batch = 30.0

        # === ИСПРАВЛЕНИЕ ЗДЕСЬ ===
        if np.isfinite(loss_value):
            running_loss += loss_value * batch_size
        else:
            running_loss += 30.0 * batch_size # Штраф за NaN/Inf loss
        # ========================

        total_lev_dist += lev_dist_batch * batch_size; total_samples += batch_size; total_lr += current_lr; num_batches_processed += 1
        pbar.set_postfix(loss=f'{loss_value:.3f}', lev=f'{lev_dist_batch:.3f}', lr=f'{current_lr:.2e}')
    pbar.close()
    avg_loss = running_loss / total_samples if total_samples > 0 else float('inf')
    avg_lev = total_lev_dist / total_samples if total_samples > 0 else float('inf')
    avg_lr = total_lr / num_batches_processed if num_batches_processed > 0 else 0.0
    return avg_loss, avg_lev, avg_lr

# --- Функция Валидации Одной Эпохи ---
def validate_epoch(model: nn.Module, dataloader: DataLoader, criterion: nn.CTCLoss, device: torch.device,
                   index_to_char_map: Dict[int, str], blank_idx: int, pad_idx: int
                  ) -> Tuple[float, float, List[Tuple[str, str]]]:
    """ Выполняет одну эпоху валидации. """
    model.eval(); running_loss = 0.0; total_lev_dist = 0.0; total_samples = 0; all_decoded_pairs = []
    try: time_factor = max(model.get_time_reduction_factor(), 1.0)
    except AttributeError: time_factor = 1.0
    pbar = tqdm(dataloader, desc="   [Валидация]", leave=False, ncols=1000)
    with torch.no_grad():
        for batch_data in pbar:
            if batch_data is None: continue
            features, targets, feature_lengths, target_lengths = batch_data
            if features is None or targets is None: continue
            batch_size = features.size(0)
            if batch_size == 0: continue

            features = features.to(device, non_blocking=True); targets = targets.to(device, non_blocking=True)
            feature_lengths_cpu = feature_lengths.cpu(); target_lengths_cpu = target_lengths.cpu()
            input_lengths_ctc = torch.floor(feature_lengths_cpu.float() / time_factor + 1e-9).long().clamp(min=1)
            loss_value = float('inf'); lev_dist_batch = float('inf'); decoded_pairs_batch = []
            try:
                logits = model(features); output_length = logits.shape[0]
                if logits.shape[1] != batch_size: print(f"\n!!! ERROR (Val): Batch size mismatch! Out:{logits.shape[1]} != In:{batch_size}. Skip."); continue
                log_probs = F.log_softmax(logits, dim=2)
                input_lengths_ctc_clamped = input_lengths_ctc.clamp(max=output_length)
                target_lengths_clamped = target_lengths_cpu.clamp(max=targets.shape[1])
                valid_target_mask = target_lengths_clamped > 0
                if not torch.all(valid_target_mask):
                    log_probs_valid = log_probs[:, valid_target_mask, :]; targets_valid = targets[valid_target_mask, :]
                    input_lengths_valid = input_lengths_ctc_clamped[valid_target_mask]; target_lengths_valid = target_lengths_clamped[valid_target_mask]
                    loss = criterion(log_probs_valid, targets_valid, input_lengths_valid, target_lengths_valid) if log_probs_valid.shape[1] > 0 else torch.tensor(0.0, device=device)
                else: loss = criterion(log_probs, targets, input_lengths_ctc_clamped, target_lengths_clamped)

                # === ИСПРАВЛЕНИЕ ЗДЕСЬ (на всякий случай, хотя здесь не было ошибки) ===
                if torch.isfinite(loss):
                    loss_value = loss.item()
                else:
                    print("\nWarning (Val): NaN/Inf loss.")
                    loss_value = float('inf') # Оставляем inf для валидации
                # =======================================================================

                if 'ctc_greedy_decode' not in globals(): raise NameError("ctc_greedy_decode missing")
                decoded_preds = ctc_greedy_decode(logits, index_to_char_map, blank_idx)
                if 'calculate_levenshtein' not in globals(): raise NameError("calculate_levenshtein missing")
                lev_dist_batch, decoded_pairs_batch = calculate_levenshtein(decoded_preds, targets.cpu(), target_lengths_cpu, index_to_char_map, pad_idx)
                if not np.isfinite(lev_dist_batch): lev_dist_batch = 20.0
            except Exception as e: print(f"\n!!! Error (Val): {e} !!!"); traceback.print_exc(limit=1); loss_value=float('inf'); lev_dist_batch=float('inf'); decoded_pairs_batch = [("ERROR","ERROR")] * batch_size

            # === ИСПРАВЛЕНИЕ ЗДЕСЬ ===
            if np.isfinite(loss_value): # Используем numpy isfinite для единообразия
                running_loss += loss_value * batch_size
            # Если loss inf, то running_loss не увеличивается, что логично для валидации
            # ========================

            total_lev_dist += lev_dist_batch * batch_size; total_samples += batch_size
            if len(all_decoded_pairs) < 10: all_decoded_pairs.extend(decoded_pairs_batch[:max(0, 10 - len(all_decoded_pairs))])
            pbar.set_postfix(loss=f'{loss_value:.3f}', lev=f'{lev_dist_batch:.3f}')
    pbar.close()
    avg_loss = running_loss / total_samples if total_samples > 0 else float('inf')
    avg_lev = total_lev_dist / total_samples if total_samples > 0 else float('inf')
    return avg_loss, avg_lev, all_decoded_pairs

print("Функции train_epoch и validate_epoch определены (SyntaxError исправлен).")
print("\n--- Ячейка 10: Определение функций обучения и валидации завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 10: Определение функций обучения и валидации эпохи ---
Функции train_epoch и validate_epoch определены (SyntaxError исправлен).

--- Ячейка 10: Определение функций обучения и валидации завершено ---
--------------------------------------------------


### Ячейка 11 Интерактивная настройка RMS

In [11]:
# =============================================================================
# Ячейка 11: Интерактивная Настройка RMS (Исправлена + Упрощена)
# =============================================================================
print(f"--- Ячейка 11: Интерактивная Настройка RMS (SR={AUDIO_CONFIG['sample_rate']} Гц) ---")

if not IPYWIDGETS_AVAILABLE:
    print("ipywidgets не доступен. Пропуск интерактивной ячейки.")
else:
    # Импорты
    import ipywidgets as widgets
    from IPython.display import display, Audio, clear_output
    import matplotlib.pyplot as plt
    import librosa, librosa.display, numpy as np, pandas as pd
    from pathlib import Path; import traceback, random
    # Добавим signal из scipy, если его нет глобально к этому моменту
    if 'signal' not in globals(): import scipy.signal as signal

    # Проверка глобальных переменных
    required_globals_vis = ['AUDIO_CONFIG', 'EXTRACTED_AUDIO_DIR', 'train_df', 'SEED']
    missing_globals_vis = [v for v in required_globals_vis if v not in globals()]
    if missing_globals_vis: raise NameError(f"Отсутствуют переменные: {missing_globals_vis}")
    if train_df.empty: raise ValueError("train_df пуст.")

    # Параметры из базового конфига
    SR_VIS = AUDIO_CONFIG['sample_rate']
    APPLY_FILTER_VIS = AUDIO_CONFIG.get('apply_filter', True)
    FILTER_ORDER_VIS = AUDIO_CONFIG.get('filter_order', 5)
    FILTER_FMIN_VIS = AUDIO_CONFIG.get('filter_fmin', 0)
    FILTER_FMAX_VIS = AUDIO_CONFIG.get('filter_fmax', SR_VIS/2*0.999)
    # Нормализация всегда Z-Score, Delta всегда standard
    print(f"Параметры визуализации: SR={SR_VIS}Hz, Filter={APPLY_FILTER_VIS}, Norm=Z-Score")

    # --- Упрощенная функция расчета признаков для визуализации ---
    def calculate_interactive_features_simple(waveform_np: np.ndarray, sr: int,
                                              frame_len_samples: int, hop_len_samples: int,
                                              apply_filter: bool, filter_order: int, fmin: float, fmax: float,
                                              file_id_vis: str = "N/A") -> Tuple[np.ndarray, np.ndarray]:
        """ Расчет RMS и Delta с Z-Score нормализацией для визуализации. """
        rms_norm_res = np.array([], dtype=np.float32); delta_norm_res = np.array([], dtype=np.float32)
        if not isinstance(waveform_np, np.ndarray) or waveform_np.size < frame_len_samples: return rms_norm_res, delta_norm_res
        try:
            processed_waveform = waveform_np.astype(np.float32)
            # 1. Фильтрация (если включена)
            if apply_filter:
                nyquist_vis = sr / 2.0; fmax_eff = min(fmax, nyquist_vis * 0.999); fmin_eff = max(0.0, fmin)
                is_lowcut = fmin_eff > 1e-3; is_highcut = fmax_eff < (nyquist_vis * 0.995)
                if is_lowcut or is_highcut:
                    try:
                        if is_lowcut and is_highcut: ftype, freqs = 'bandpass', [fmin_eff, fmax_eff]
                        elif is_lowcut: ftype, freqs = 'highpass', fmin_eff
                        else: ftype, freqs = 'lowpass', fmax_eff
                        if not (isinstance(freqs, list) and freqs[0] >= freqs[1]):
                            sos = signal.butter(filter_order, freqs, btype=ftype, fs=sr, output='sos')
                            processed_waveform = signal.sosfiltfilt(sos, processed_waveform)
                    except Exception as filter_e: print(f"Warning ({file_id_vis}): Ошибка фильтрации виз: {filter_e}")
            # 2. RMS
            rms_env = librosa.feature.rms(y=processed_waveform, frame_length=frame_len_samples, hop_length=hop_len_samples, center=True, pad_mode='reflect')[0]
            if rms_env.size < 2: return rms_norm_res, delta_norm_res
            # 3. Delta (standard)
            delta_feat = np.diff(rms_env, n=1, prepend=rms_env[0])
            # 4. Нормализация Z-Score
            epsilon = 1e-8
            mean_rms = np.mean(rms_env); std_rms = np.std(rms_env); rms_norm_res = ((rms_env - mean_rms) / (std_rms + epsilon)).astype(np.float32) if std_rms >= epsilon else np.zeros_like(rms_env)
            mean_del = np.mean(delta_feat); std_del = np.std(delta_feat); delta_norm_res = ((delta_feat - mean_del) / (std_del + epsilon)).astype(np.float32) if std_del >= epsilon else np.zeros_like(delta_feat)
            return rms_norm_res, delta_norm_res
        except Exception as e: print(f"Error calc interactive features ({file_id_vis}): {e}"); traceback.print_exc(limit=1); return np.array([]), np.array([])

    # --- Функция отрисовки графиков ---
    def plot_interactive_rms_simple(times: np.ndarray, waveform: np.ndarray, rms_norm: np.ndarray, delta_norm: np.ndarray,
                                    sr: int, frame_len_samples: int, hop_len_samples: int, file_id: str, frame_ms: float, hop_ms: float):
        fig, axes = plt.subplots(4, 1, figsize=(16, 10), sharex=True)
        title = f"Интерактивный Анализ RMS (Z-Score): {file_id}\nSR={sr}, Frame={frame_len_samples} samples (~{frame_ms:.1f}ms), Hop={hop_len_samples} samples (~{hop_ms:.1f}ms)"
        fig.suptitle(title, fontsize=14)
        librosa.display.waveshow(waveform, sr=sr, ax=axes[0], color='grey', alpha=0.7); axes[0].set_title("Waveform"); axes[0].label_outer(); axes[0].grid(True, linestyle=':')
        if times.size > 0 and rms_norm.size == times.size and delta_norm.size == times.size:
            axes[1].plot(times, rms_norm, label='RMS (Z-Score)', color='blue', linewidth=1.5); axes[1].set_title("Норм. RMS (Z-Score)"); axes[1].legend(); axes[1].grid(True, linestyle=':'); axes[1].label_outer()
            axes[2].plot(times, delta_norm, label='Delta (Z-Score)', color='red', linewidth=1.5); axes[2].axhline(0, color='black', linestyle=':', linewidth=1); axes[2].set_title("Норм. Delta (Z-Score)"); axes[2].legend(); axes[2].grid(True, linestyle=':'); axes[2].label_outer()
            img = axes[3].imshow(rms_norm.reshape(1, -1), aspect='auto', cmap='magma', interpolation='nearest', extent=[times.min(), times.max(), 0, 1]); axes[3].set_title("Норм. RMS (Heatmap)"); axes[3].set_yticks([])
            fig.colorbar(img, ax=axes[3], orientation='horizontal', label='Норм. RMS (Z-Score)', pad=0.2)
        else:
            for i in range(1, 4): axes[i].set_title(f"График {i+1} - Нет данных"); axes[i].text(0.5, 0.5, 'Нет данных', ha='center', va='center', transform=axes[i].transAxes); axes[i].label_outer()
        axes[-1].set_xlabel("Время (с)"); plt.tight_layout(rect=[0, 0.03, 1, 0.94]); plt.show()

    # --- Виджеты ---
    file_ids_vis_list = train_df['id'].unique().tolist(); file_ids_vis_list = random.sample(file_ids_vis_list, min(500, len(file_ids_vis_list)))
    if not file_ids_vis_list: raise SystemExit("Остановка: Нет файлов для визуализации.")
    file_dd = widgets.Dropdown(options=file_ids_vis_list, description='Аудиофайл:', style={'description_width': 'initial'})
    frame_ms_slider = widgets.IntSlider(value=24, min=8, max=64, step=2, description='Frame (ms):', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px'), continuous_update=False)
    hop_ms_slider = widgets.IntSlider(value=12, min=4, max=24, step=1, description='Hop (ms):', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px'), continuous_update=False)
    samples_label = widgets.Label(value="")
    params_box = widgets.VBox([frame_ms_slider, hop_ms_slider, samples_label])
    plot_output = widgets.Output()
    def _link_sliders_ms(change): frame_ms = frame_ms_slider.value; hop_ms_slider.max = frame_ms; hop_ms_slider.value = min(hop_ms_slider.value, frame_ms)
    frame_ms_slider.observe(_link_sliders_ms, names='value')

    # --- Обработчик (С ИСПРАВЛЕНИЕМ) ---
    def handle_rms_vis_change_simple(change):
        frame_ms = frame_ms_slider.value; hop_ms = hop_ms_slider.value; file_id = file_dd.value
        frame_samples = max(1, int(frame_ms * SR_VIS / 1000)); hop_samples = max(1, int(hop_ms * SR_VIS / 1000))
        if frame_samples < hop_samples: frame_samples = hop_samples
        samples_label.value = f"(Frame: {frame_samples} сэмплов, Hop: {hop_samples} сэмплов)"

        # === ИСПРАВЛЕНИЕ ЗДЕСЬ ===
        if not file_id:
            with plot_output: # Отдельный блок with
                clear_output(wait=True)
                print("Выберите файл.")
            return # Выход из функции
        # ========================

        audio_path = EXTRACTED_AUDIO_DIR / file_id;
        # === ИСПРАВЛЕНИЕ ЗДЕСЬ ===
        if not audio_path.is_file():
            with plot_output: # Отдельный блок with
                clear_output(wait=True)
                print(f"Ошибка: Файл не найден {audio_path}")
            return # Выход из функции
        # ========================

        try:
            waveform, _ = librosa.load(audio_path, sr=SR_VIS, mono=True)
            # Расчет УПРОЩЕННЫХ признаков
            rms_norm_res, delta_norm_res = calculate_interactive_features_simple(
                waveform, SR_VIS, frame_samples, hop_samples,
                APPLY_FILTER_VIS, FILTER_ORDER_VIS, FILTER_FMIN_VIS, FILTER_FMAX_VIS,
                file_id_vis=file_id )
            times_res = librosa.times_like(rms_norm_res, sr=SR_VIS, hop_length=hop_samples) if rms_norm_res.size > 0 else np.array([])
            with plot_output:
                clear_output(wait=True); print(f"Отображение: {file_id} (SR={SR_VIS}Hz)")
                # Вызов УПРОЩЕННОЙ функции отрисовки
                plot_interactive_rms_simple(times_res, waveform, rms_norm_res, delta_norm_res, SR_VIS, frame_samples, hop_samples, file_id, frame_ms, hop_ms)
        except Exception as e:
             # === ИСПРАВЛЕНИЕ ЗДЕСЬ (на всякий случай) ===
            with plot_output: # Отдельный блок with
                clear_output(wait=True)
                print(f"Ошибка обработки {file_id}:\n{e}")
                traceback.print_exc(limit=2)
            # ========================

    # --- Привязка и отображение ---
    file_dd.observe(handle_rms_vis_change_simple, names='value')
    frame_ms_slider.observe(handle_rms_vis_change_simple, names='value')
    hop_ms_slider.observe(handle_rms_vis_change_simple, names='value')
    ui_rms_tuning = widgets.VBox([ widgets.HTML(f"<b>Интерактивная настройка RMS (SR={SR_VIS}Hz, Z-Score)</b>"), file_dd, params_box, plot_output ])
    display(ui_rms_tuning)
    _link_sliders_ms(None); handle_rms_vis_change_simple(None) # Первый запуск

    print(f"\n--- Ячейка 11: Интерактивная настройка RMS (Исправлена) готова ---")
print("-" * 50)
# =============================================================================

--- Ячейка 11: Интерактивная Настройка RMS (SR=8000 Гц) ---
Параметры визуализации: SR=8000Hz, Filter=False, Norm=Z-Score



--- Ячейка 11: Интерактивная настройка RMS (Исправлена) готова ---
--------------------------------------------------


### Ячейка 12: Функция Полного Цикла Обучения

In [12]:
# =============================================================================
# Ячейка 12: Функция для Полного Цикла Обучения и Инференса (Исправлена заглушка MLflow)
# =============================================================================
print("--- Ячейка 12: Определение функции run_training_pipeline ---")

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torch.optim.lr_scheduler import OneCycleLR
import numpy as np, pandas as pd, time, json, random, traceback
from pathlib import Path
from tqdm.notebook import tqdm
from typing import Union, Tuple, List, Dict, Optional

# --- Проверка и импорт MLflow или создание ЗАГЛУШКИ (ИСПРАВЛЕНО) ---
try:
    import mlflow
    import mlflow.pytorch
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False
    print("Предупреждение: mlflow не найден. Логирование экспериментов будет отключено.")
    class DummyRun:
        def __enter__(self): return self
        def __exit__(self, *args): pass
        class DummyRunInfo: info = type('obj', (object,), {'run_id': 'mlflow_disabled'})()
        info = DummyRunInfo()
    class DummyMLflow:
        def set_experiment(self, *args, **kwargs): pass
        def start_run(self, *args, **kwargs): return DummyRun()
        def log_param(self, *args, **kwargs): pass
        def log_params(self, *args, **kwargs): pass
        # === ИСПРАВЛЕНИЕ ЗДЕСЬ: Каждая функция на своей строке ===
        def log_metric(self, *args, **kwargs): pass
        def log_metrics(self, *args, **kwargs): pass
        # =====================================================
        def log_artifact(self, *args, **kwargs): pass
        def log_dict(self, *args, **kwargs): pass
        def set_tag(self, *args, **kwargs): pass
        def active_run(self): return None
        def end_run(self, status='FINISHED'): pass
    mlflow = DummyMLflow()


# --- Проверка глобальных зависимостей ---
def check_global_dependencies():
    required = ['MorseDataset', 'MorseRecognizer', 'collate_fn', 'train_epoch', 'validate_epoch',
                'ctc_greedy_decode', 'calculate_levenshtein', 'set_seed']
    missing = [r for r in required if not callable(globals().get(r))]
    if missing: raise NameError(f"Отсутствуют зависимости: {missing}. Убедитесь, что все ячейки выше выполнены.")

# --- Функция пайплайна ---
def run_training_pipeline(
    # Конфиги
    audio_config: Dict, model_config: Dict, train_config: Dict,
    # Данные/пути
    full_train_df: pd.DataFrame, test_df: pd.DataFrame, audio_dir: Path, base_output_dir: Path,
    # Словари/константы
    char_to_index: Dict[str, int], index_to_char: Dict[int, str],
    blank_idx: int, pad_idx: int, num_classes_ctc: int,
    # Управление
    device: torch.device, run_suffix: str, base_filename_suffix: str, # Используем полный базовый суффикс
    seed: int = 42, mlflow_experiment_name: str = "Morse_Experiment"
) -> Dict:
    """ Выполняет полный цикл: данные, обучение, сохранение, инференс. """
    check_global_dependencies()
    run_results = { "run_suffix": run_suffix, "status": "pending", "best_val_lev": float('inf'), "best_epoch": None,
                    "final_train_loss": float('inf'), "final_val_loss": float('inf'), "train_time_min": 0.0,
                    "infer_time_sec": 0.0, "model_path": None, "params_path": None, "submission_path": None, "error": None }
    print(f"\n{'='*20} Запуск: {run_suffix} {'='*20}")
    print(f"Audio Config: {json.dumps(audio_config)}")
    print(f"Model Config: {json.dumps(model_config)}") # Выводим конфиг модели
    print(f"Train Config: {json.dumps(train_config)}")

    # Формирование путей (используем полный базовый суффикс + суффикс запуска)
    output_filename_base = f"{base_filename_suffix}{run_suffix}"
    current_model_path = base_output_dir / f"model_{output_filename_base}.pth"
    current_params_path = base_output_dir / f"params_{output_filename_base}.json"
    current_submission_path = base_output_dir / f"submission_greedy_{output_filename_base}.csv"
    run_results.update({ "model_path": str(current_model_path), "params_path": str(current_params_path), "submission_path": str(current_submission_path) })
    print(f"  Пути: Model={current_model_path.name}, Params={current_params_path.name}, Sub={current_submission_path.name}")

    # Объявление переменных для finally
    model = None; criterion = None; optimizer = None; scheduler = None
    full_dataset = None; train_subset = None; val_subset = None
    train_loader = None; val_loader = None
    inference_model_instance = None; infer_test_dataset = None; test_loader_infer = None
    logits_infer = None; features_infer = None

    try:
        set_seed(seed)
        # --- 1. Подготовка Данных ---
        print("\n1. Подготовка данных...")
        if 'MorseDataset' not in globals(): raise NameError("Класс MorseDataset не определен!")
        full_dataset = MorseDataset( dataframe=full_train_df, audio_dir=audio_dir, char_to_index=char_to_index,
                                     audio_config=audio_config, model_input_feature_dim=model_config['input_feature_dim'],
                                     is_train=True )
        dataset_size = len(full_dataset);
        if dataset_size == 0: raise ValueError("Обучающий датасет пуст!")
        val_split_ratio = train_config['validation_split_ratio']
        val_size = int(np.floor(val_split_ratio * dataset_size)); train_size = dataset_size - val_size
        if train_size <= 0 or val_size <= 0: raise ValueError(f"Некорректное разделение: Train={train_size}, Val={val_size}")
        generator = torch.Generator().manual_seed(seed)
        train_subset, val_subset = random_split(full_dataset, [train_size, val_size], generator=generator)
        bs = train_config['batch_size']; nw = train_config['num_workers']; pm = (device.type == 'cuda')
        if 'collate_fn' not in globals(): raise NameError("Функция collate_fn не определена!")
        train_loader = DataLoader(train_subset, batch_size=bs, shuffle=True, collate_fn=collate_fn, num_workers=nw, pin_memory=pm)
        val_loader = DataLoader(val_subset, batch_size=bs*2, shuffle=False, collate_fn=collate_fn, num_workers=nw, pin_memory=pm)
        print(f"  Данные готовы: Train={len(train_subset)} ({len(train_loader)} батчей), Val={len(val_subset)} ({len(val_loader)} батчей).")

        # --- 2. Инициализация Модели, Loss, Optimizer, Scheduler ---
        print("\n2. Инициализация компонентов...")
        if 'MorseRecognizer' not in globals(): raise NameError("Класс MorseRecognizer не определен!")
        model = MorseRecognizer(num_classes_ctc=num_classes_ctc, **model_config).to(device) # Используем переданный model_config
        criterion = nn.CTCLoss(blank=blank_idx, reduction='mean', zero_infinity=True).to(device)
        optimizer_name = train_config.get('optimizer', 'AdamW').lower(); lr = train_config['learning_rate']; wd = train_config['weight_decay']
        if optimizer_name == 'adamw': optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        elif optimizer_name == 'adam': optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        else: print(f"Warning: Неизвестный optimizer '{optimizer_name}'. Используется AdamW."); optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        print(f"  Model, Loss, Optimizer ({type(optimizer).__name__}) готовы.")
        # Scheduler
        total_batches_in_loader = len(train_loader); batches_per_epoch_run = train_config.get("batches_per_epoch", 0)
        steps_per_epoch = total_batches_in_loader
        if batches_per_epoch_run > 0 and batches_per_epoch_run < total_batches_in_loader: steps_per_epoch = batches_per_epoch_run
        if steps_per_epoch <= 0: raise ValueError(f"Steps_per_epoch ({steps_per_epoch}) должен быть > 0!")
        total_steps = train_config['num_epochs'] * steps_per_epoch
        div_f = train_config.get('div_factor', 25.0); final_div_f = train_config.get('final_div_factor', 1e4)
        scheduler = OneCycleLR(optimizer, max_lr=lr, total_steps=total_steps, pct_start=0.3, anneal_strategy='cos', div_factor=div_f, final_div_factor=final_div_f)
        print(f"  Scheduler OneCycleLR создан (total_steps={total_steps}, div_factor={div_f}, final_div_factor={final_div_f}).")

        # --- 3. Цикл Обучения с MLflow ---
        print("\n3. Запуск цикла обучения...")
        mlflow.set_experiment(mlflow_experiment_name)
        with mlflow.start_run(run_name=f"run_{run_suffix}") as current_run:
            mlflow_run_id = current_run.info.run_id; print(f"  MLflow Run ID: {mlflow_run_id}")
            run_results["mlflow_run_id"] = mlflow_run_id; run_results["status"] = "training"
            mlflow.log_param("run_suffix", run_suffix); mlflow.log_param("base_filename", base_filename_suffix)
            mlflow.log_params({"seed": seed, **train_config}); mlflow.log_dict(audio_config, "audio_config.json"); mlflow.log_dict(model_config, "model_config.json")
            mlflow.set_tag("status", "training")

            start_time_train = time.time(); best_val_lev = float('inf'); epochs_without_improvement = 0; best_epoch = None
            best_model_local_path_temp = base_output_dir / f"temp_best_model_{mlflow_run_id}.pth"

            for epoch in range(1, train_config['num_epochs'] + 1):
                print(f"\n--- Эпоха {epoch}/{train_config['num_epochs']} ---")
                if 'train_epoch' not in globals(): raise NameError("Функция train_epoch не определена!")
                avg_train_loss, avg_train_lev, avg_epoch_lr = train_epoch( model, train_loader, criterion, optimizer, scheduler, device, epoch, train_config['num_epochs'], index_to_char, blank_idx, pad_idx, train_config['gradient_clip_norm'], batches_per_epoch=batches_per_epoch_run )
                if 'validate_epoch' not in globals(): raise NameError("Функция validate_epoch не определена!")
                avg_val_loss, avg_val_lev, _ = validate_epoch( model, val_loader, criterion, device, index_to_char, blank_idx, pad_idx )
                run_results.update({ "final_train_loss": avg_train_loss, "final_val_loss": avg_val_loss })
                mlflow.log_metrics({ "train_loss": avg_train_loss, "train_levenshtein": avg_train_lev, "val_loss": avg_val_loss, "val_levenshtein": avg_val_lev, "learning_rate": avg_epoch_lr }, step=epoch)
                print(f"  Итоги Эпохи {epoch}: Train Loss={avg_train_loss:.4f}, Train Lev={avg_train_lev:.4f}, Val Loss={avg_val_loss:.4f}, Val Lev={avg_val_lev:.4f}")

                if np.isfinite(avg_val_lev) and avg_val_lev < best_val_lev:
                    print(f"  ✨ Val Lev улучшился: {best_val_lev:.4f} -> {avg_val_lev:.4f}. Сохранение..."); best_val_lev = avg_val_lev; best_epoch = epoch
                    run_results.update({ "best_val_lev": best_val_lev, "best_epoch": best_epoch })
                    try: torch.save(model.state_dict(), best_model_local_path_temp)
                    except Exception as save_err: print(f"  !!! Ошибка сохранения временной модели: {save_err} !!!")
                    epochs_without_improvement = 0
                else:
                    epochs_without_improvement += 1; print(f"  Val Lev не улучшился ({avg_val_lev:.4f} vs best {best_val_lev:.4f}). Без улучшений: {epochs_without_improvement}/{train_config['early_stopping_patience']}")
                if epochs_without_improvement >= train_config['early_stopping_patience']:
                    print(f"  ❗️ Ранняя остановка на эпохе {epoch}!"); mlflow.set_tag("status", "completed_early_stopping"); run_results["status"] = "completed_early_stopping"; break
            # --- Конец цикла по эпохам ---
            if run_results["status"] == "training": run_results["status"] = "completed"; mlflow.set_tag("status", "completed")
            train_duration_min = (time.time() - start_time_train) / 60; run_results["train_time_min"] = train_duration_min
            print(f"\n  Обучение завершено ({run_results['status']}) за {train_duration_min:.2f} мин. Лучший Val Lev: {best_val_lev:.4f} (Эпоха {best_epoch})")

            # --- 4. Сохранение Лучшей Модели и Параметров ---
            print("\n4. Сохранение артефактов...")
            if np.isfinite(best_val_lev): mlflow.log_metric("best_val_levenshtein", best_val_lev)
            if best_epoch is not None: mlflow.log_metric("best_epoch", best_epoch)
            mlflow.log_metric("training_time_min", train_duration_min)

            model_saved_final = False
            if best_model_local_path_temp.exists() and np.isfinite(best_val_lev):
                try:
                    current_model_path.parent.mkdir(parents=True, exist_ok=True); current_model_path.unlink(missing_ok=True)
                    best_model_local_path_temp.rename(current_model_path); print(f"  Лучшая модель сохранена: {current_model_path.name}")
                    mlflow.log_artifact(str(current_model_path), artifact_path="model"); model_saved_final = True
                except Exception as e: print(f"  !!! Ошибка перемещения/логирования модели: {e} !!!")
            else: print("  !!! Лучшая модель не сохранена."); mlflow.set_tag("model_saved", "False")

            final_params = { 'audio_config': audio_config, 'model_config': model_config, 'train_config': train_config,
                             'char_map': { 'char_to_index': char_to_index, 'index_to_char': {str(k): v for k, v in index_to_char.items()},
                                           'BLANK_IDX': blank_idx, 'PAD_IDX': pad_idx, 'NUM_CLASSES_CTC': num_classes_ctc },
                             'results': { k: v for k, v in run_results.items() if k not in ['model_path', 'params_path', 'submission_path', 'mlflow_run_id'] and (np.isfinite(v) if isinstance(v, float) else True) } }
            try:
                current_params_path.parent.mkdir(parents=True, exist_ok=True); current_params_path.unlink(missing_ok=True)
                with open(current_params_path, 'w', encoding='utf-8') as f: json.dump(final_params, f, indent=4, ensure_ascii=False)
                print(f"  Параметры сохранены: {current_params_path.name}"); mlflow.log_artifact(str(current_params_path), artifact_path="config")
            except Exception as e: print(f"  !!! Ошибка сохранения/логирования параметров: {e} !!!")
        # --- Конец MLflow run ---

        # --- 5. Инференс на Тестовых Данных ---
        print("\n5. Запуск инференса...")
        infer_duration_sec = 0.0
        if model_saved_final:
            infer_start_time = time.time()
            try:
                 print(f"  Загрузка: {current_model_path.name}, {current_params_path.name}")
                 with open(current_params_path, 'r', encoding='utf-8') as f: loaded_params_inf = json.load(f)
                 loaded_audio_config_inf = loaded_params_inf['audio_config']; loaded_model_config_inf = loaded_params_inf['model_config']
                 loaded_char_map_inf = loaded_params_inf['char_map']; loaded_index_to_char_inf = {int(k): v for k, v in loaded_char_map_inf['index_to_char'].items()}
                 loaded_blank_idx_inf = loaded_char_map_inf['BLANK_IDX']; loaded_num_classes_inf = loaded_char_map_inf['NUM_CLASSES_CTC']

                 if 'MorseRecognizer' not in globals(): raise NameError("Класс MorseRecognizer не определен для инференса!")
                 inference_model_instance = MorseRecognizer(num_classes_ctc=loaded_num_classes_inf, **loaded_model_config_inf).to(device)
                 try: inference_model_instance.load_state_dict(torch.load(current_model_path, map_location=device, weights_only=True))
                 except TypeError: print("   Warning: weights_only=True не поддерживается."); inference_model_instance.load_state_dict(torch.load(current_model_path, map_location=device))
                 inference_model_instance.eval(); print(f"  Модель для инференса загружена.")

                 print(f"  Создание тестового DataLoader...");
                 if 'MorseDataset' not in globals(): raise NameError("Класс MorseDataset не определен для инференса!")
                 infer_test_dataset = MorseDataset( dataframe=test_df, audio_dir=audio_dir, char_to_index=loaded_char_map_inf['char_to_index'],
                                                    audio_config=loaded_audio_config_inf, model_input_feature_dim=loaded_model_config_inf['input_feature_dim'], is_train=False )
                 infer_bs = train_config.get('batch_size', 16) * 4
                 if 'collate_fn' not in globals(): raise NameError("Функция collate_fn не определена для инференса!")
                 test_loader_infer = DataLoader(infer_test_dataset, batch_size=infer_bs, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=(device.type == 'cuda') )

                 print(f"  Предсказание..."); predictions: Dict[str, str] = {}
                 if 'ctc_greedy_decode' not in globals(): raise NameError("Функция ctc_greedy_decode не определена для инференса!")
                 with torch.no_grad():
                     pbar_infer = tqdm(test_loader_infer, desc="Инференс", leave=False, ncols=1000)
                     for batch_data_infer in pbar_infer:
                         if batch_data_infer is None: continue
                         features_infer, file_ids_infer, _, _ = batch_data_infer
                         if features_infer is None or file_ids_infer is None or len(features_infer) == 0: continue
                         features_infer = features_infer.to(device, non_blocking=True)
                         try:
                             logits_infer = inference_model_instance(features_infer)
                             decoded_batch_infer = ctc_greedy_decode(logits_infer, loaded_index_to_char_inf, loaded_blank_idx_inf)
                             for file_id, pred_text in zip(file_ids_infer, decoded_batch_infer): predictions[file_id] = pred_text
                         except Exception as e_inf_batch: print(f"\nОшибка инференса батча: {e_inf_batch}"); [predictions.update({fid:"ERROR_INFER"}) for fid in file_ids_infer]

                 infer_duration_sec = time.time() - infer_start_time; run_results["infer_time_sec"] = infer_duration_sec
                 print(f"  Инференс завершен за {infer_duration_sec:.2f} сек. Предсказаний: {len(predictions)}/{len(test_df)}")

                 if predictions:
                     print(f"  Формирование submission: {current_submission_path.name}")
                     submission_df = pd.DataFrame({'id': test_df['id']}); submission_df['message'] = submission_df['id'].map(predictions).fillna("ERROR_MISSING")
                     current_submission_path.unlink(missing_ok=True)
                     submission_df.to_csv(current_submission_path, index=False); print(f"  Submission сохранен."); mlflow.log_artifact(str(current_submission_path), artifact_path="submission")
                 else: print("  Предсказания не сгенерированы."); mlflow.set_tag("submission_generated", "False")
            except Exception as e_infer: print(f"  !!! Ошибка инференса: {e_infer} !!!"); traceback.print_exc(limit=2); run_results["error"] = f"Inference Error: {e_infer}"; run_results["status"] = "failed_inference"; mlflow.set_tag("status", "failed_inference")
            finally:
                 try: del inference_model_instance
                 except NameError: pass
                 try: del test_loader_infer
                 except NameError: pass
                 try: del infer_test_dataset
                 except NameError: pass
                 if 'logits_infer' in locals(): del logits_infer
                 if 'features_infer' in locals(): del features_infer
                 if torch.cuda.is_available(): torch.cuda.empty_cache()
        else: print("  Инференс пропущен (лучшая модель не сохранена)."); run_results["status"] += "_no_inference"; mlflow.set_tag("inference_skipped", "True")

    except Exception as e_main: print(f"!!! КРИТИЧЕСКАЯ ОШИБКА в {run_suffix}: {e_main} !!!"); traceback.print_exc(); run_results["error"] = str(e_main); run_results["status"] = "failed_critical";
    try: mlflow.set_tag("status", "failed_critical"); mlflow.set_tag("error", str(e_main)[:250])
    except: pass

    finally:
        try: del model
        except NameError: pass
        try: del criterion
        except NameError: pass
        try: del optimizer
        except NameError: pass
        try: del scheduler
        except NameError: pass
        if 'full_dataset' in locals(): del full_dataset
        if 'train_subset' in locals(): del train_subset
        if 'val_subset' in locals(): del val_subset
        if 'train_loader' in locals(): del train_loader
        if 'val_loader' in locals(): del val_loader
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        print(f"{'='*20} Завершение: {run_suffix} {'='*20}\n")
    return run_results

print("--- Функция run_training_pipeline (Исправлена заглушка MLflow) определена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 12: Определение функции run_training_pipeline ---
--- Функция run_training_pipeline (Исправлена заглушка MLflow) определена ---
--------------------------------------------------


### Ячейка 13: Основной Цикл Обучения

In [13]:
# =============================================================================
# Ячейка 13: Цикл Запуска Обучения (Тест Double Classifier для лучшей модели 3x512)
# =============================================================================
print(f"--- Ячейка 13: Запуск Цикла Обучения (Тест Double Classifier для лучшей модели 3x512) ---")

import time, pandas as pd, numpy as np, json, random, copy, traceback
from pathlib import Path; import torch
from IPython.display import display

# --- Проверка глобальных зависимостей ---
# Убедимся, что все нужное есть, включая BASE_FILENAME_SUFFIX_FINAL
required_globals_loop = [ 'train_df', 'test_df', 'EXTRACTED_AUDIO_DIR', 'char_to_index', 'index_to_char', 'BLANK_IDX', 'PAD_IDX', 'NUM_CLASSES_CTC', 'device', 'OUTPUT_DIR', 'BASE_FILENAME_SUFFIX_FINAL', 'SEED', 'run_training_pipeline', 'AUDIO_CONFIG', 'MODEL_CONFIG_BASE', 'TRAIN_CONFIG', 'mlflow' ]
missing_globals_loop = [v for v in required_globals_loop if v not in globals() or globals().get(v) is None]
if missing_globals_loop: raise NameError(f"Отсутствуют переменные/функции или они None: {missing_globals_loop}")

# ============================================================================
# === БЛОК 1: Определение Конфигурации для Теста Double Classifier ===
# ============================================================================
print("\n1. Определение конфигурации для теста Double Classifier...")

# Используем MODEL_CONFIG_BASE (ЛУЧШУЮ 3x512) как основу
configs_to_run = {}
audio_configs_to_run = {}

# 1. Best Model (3x512) with Double Classifier (Trimming OFF)
config_name = "BestModel_3x512_ClsDouble"
configs_to_run[config_name] = MODEL_CONFIG_BASE.copy()
configs_to_run[config_name]["classifier_type"] = "double" # <--- Ключевое изменение

audio_configs_to_run[config_name] = AUDIO_CONFIG.copy()
audio_configs_to_run[config_name]["apply_trimming"] = False # Trimming выключен

print(f"\nКонфигурация для запуска ({len(configs_to_run)}):")
m_cfg = configs_to_run[config_name]
a_cfg = audio_configs_to_run[config_name]
trim_status = f"Trim ON (db={a_cfg['trim_top_db']})" if a_cfg['apply_trimming'] else "Trim OFF"
print(f"  - {config_name}: RNN={m_cfg['rnn_num_layers']}x{m_cfg['rnn_hidden_size']}, Drop={m_cfg['dropout_rate']}, Cls={m_cfg['classifier_type']}, {trim_status}")
# ============================================================================
# === КОНЕЦ БЛОКА 1 ===
# ============================================================================

# --- 2. Копирование Базовой Конфигурации Обучения ---
print("\n2. Копирование базовой конфигурации обучения...")
# Конфиг обучения ФИКСИРОВАН (короткие эпохи)
fixed_train_config = TRAIN_CONFIG.copy()

# --- 3. Инициализация Списка для Результатов ---
all_run_results_list: List[Dict] = []

# ============================================================================
# === БЛОК 2: Цикл запуска обучения для одной конфигурации ===
# ============================================================================
print(f"\n--- Начало цикла запусков ({len(configs_to_run)} итерация) ---")
overall_start_time = time.time()
MLFLOW_EXPERIMENT_NAME = f"Morse_Final_ClsDouble_Test" # Новое имя эксперимента
print(f"MLflow эксперимент: '{MLFLOW_EXPERIMENT_NAME}'")

# --- Надежное завершение ЛЮБОГО активного MLflow run перед циклом ---
# (Код завершения MLflow run остается таким же)
print("\n--- Попытка завершить ЛЮБОЙ активный MLflow run перед циклом ---")
try:
    if mlflow.active_run():
        active_run_id = mlflow.active_run().info.run_id
        print(f"!!! Обнаружен активный MLflow run ({active_run_id}). Принудительно завершаем... !!!")
        mlflow.end_run(); time.sleep(1)
        if mlflow.active_run(): print("!!! ПРЕДУПРЕЖДЕНИЕ: MLflow run ВСЕ ЕЩЕ АКТИВЕН! !!!")
        else: print("--- Проверка: Активный MLflow run отсутствует. ---")
    else: print("--- Активный MLflow run не обнаружен. ---")
except Exception as e_check_run: print(f"Предупреждение: Ошибка при проверке/завершении активного MLflow run: {e_check_run}")
# -------------------------------------------------------------

# --- Основной цикл по конфигурациям (теперь только одна) ---
for i, config_name in enumerate(configs_to_run.keys()):

    current_model_config = configs_to_run[config_name]
    current_audio_config = audio_configs_to_run[config_name]

    print(f"\n{'='*10} Запуск {i+1}/{len(configs_to_run)}: Конфигурация '{config_name}' {'='*10}")
    print(f"  Аудио параметры: {json.dumps(current_audio_config)}")
    print(f"  Параметры модели: {json.dumps(current_model_config)}")
    print(f"{'-'*30}")

    # Формируем суффикс для имен файлов и MLflow run
    current_run_suffix = f"_{config_name.replace(' ', '_')}"

    # === Дополнительная попытка завершить run ПЕРЕД start_run ВНУТРИ цикла ===
    # (Код завершения MLflow run остается таким же)
    print(f"--- Проверка/Завершение MLflow run перед итерацией {i+1}... ---")
    try:
        if mlflow.active_run():
            lingering_run_id = mlflow.active_run().info.run_id
            print(f"!!! ПРЕДУПРЕЖДЕНИЕ: Обнаружен активный run ({lingering_run_id}) ПЕРЕД start_run в итерации {i+1}. Попытка завершения... !!!")
            mlflow.end_run(); time.sleep(0.5)
            if mlflow.active_run(): print(f"!!! ОШИБКА: Run {lingering_run_id} все еще активен! !!!")
            else: print(f"--- Зависший run {lingering_run_id} успешно завершен внутри цикла. ---")
        else: print(f"--- Активный run не найден перед итерацией {i+1}. Продолжаем. ---")
    except Exception as e_inner_end: print(f"Предупреждение: Ошибка при попытке end_run внутри цикла: {e_inner_end}")
    # ======================================================================

    # --- Вызов основной функции обучения/инференса ---
    try:
        print(f"--- Вызов run_training_pipeline для конфигурации '{config_name}' ---")
        run_result_dict = run_training_pipeline(
            audio_config=current_audio_config,
            model_config=current_model_config,
            train_config=fixed_train_config, # Короткие эпохи
            full_train_df=train_df, test_df=test_df, audio_dir=EXTRACTED_AUDIO_DIR, base_output_dir=OUTPUT_DIR,
            char_to_index=char_to_index, index_to_char=index_to_char, blank_idx=BLANK_IDX, pad_idx=PAD_IDX, num_classes_ctc=NUM_CLASSES_CTC,
            device=device,
            run_suffix=current_run_suffix,
            base_filename_suffix=BASE_FILENAME_SUFFIX_FINAL, # Используем тот же базовый суффикс
            seed=SEED,
            mlflow_experiment_name=MLFLOW_EXPERIMENT_NAME
        )
        print(f"--- Успешное завершение run_training_pipeline для '{config_name}' ---")

    except Exception as e_pipeline:
        print(f"!!! КРИТИЧЕСКАЯ ОШИБКА ПРИ ВЫЗОВЕ ПАЙПЛАЙНА для '{config_name}' !!!")
        print(f"Ошибка: {e_pipeline}"); traceback.print_exc()
        run_result_dict = { "run_suffix": current_run_suffix, "status": "failed_launch", "best_val_lev": float('inf'),
                            "error": f"Pipeline launch error: {e_pipeline}", "config_name": config_name }

    run_result_dict['config_name'] = config_name
    run_result_dict['trimming_status'] = f"ON (db={current_audio_config['trim_top_db']})" if current_audio_config['apply_trimming'] else "OFF"
    all_run_results_list.append(run_result_dict)

    print(f"--- Завершение обработки Запуска {i+1}/{len(configs_to_run)} ('{config_name}') ---")

# --- Конец цикла запусков ---
# ============================================================================
# === КОНЕЦ БЛОКА 2 ===
# ============================================================================

overall_end_time = time.time(); total_duration_hours = (overall_end_time - overall_start_time) / 3600
print(f"\n--- Цикл запусков завершен за {total_duration_hours:.2f} часов ---")

# ============================================================================
# === БЛОК 3: Анализ и вывод итогов ===
# ============================================================================
print("\n--- Итоги Теста Double Classifier ---")
if not all_run_results_list: print("Нет результатов для анализа.")
else:
    results_df = pd.DataFrame(all_run_results_list)
    required_cols = ['config_name', 'trimming_status', 'best_val_lev', 'best_epoch', 'final_train_loss', 'final_val_loss',
                     'status', 'train_time_min', 'infer_time_sec', 'error', 'run_suffix', 'mlflow_run_id'] # Убрали пути к файлам для краткости
    available_cols = [col for col in required_cols if col in results_df.columns]
    results_df = results_df[available_cols]
    if 'best_val_lev' in results_df.columns: results_df = results_df.sort_values(by='best_val_lev', ascending=True, na_position='last')
    else: print("Предупреждение: Колонка 'best_val_lev' отсутствует, сортировка невозможна.")

    pd.set_option('display.max_rows', 100); pd.set_option('display.max_columns', 20);
    pd.set_option('display.width', 180); pd.set_option('display.float_format', '{:.4f}'.format)

    print("\nСводная таблица результатов:")
    display_cols = ['config_name', 'trimming_status', 'best_val_lev', 'best_epoch', 'status', 'train_time_min', 'error']
    display_cols_present = [col for col in display_cols if col in results_df.columns]
    display(results_df[display_cols_present].head(len(results_df)))

    if 'best_val_lev' in results_df.columns and not results_df.empty:
        best_run = results_df.iloc[0]
        if pd.notna(best_run['best_val_lev']) and np.isfinite(best_run['best_val_lev']):
            print(f"\n--- Результат Теста Double Classifier ---")
            best_config_name = best_run.get('config_name', 'N/A')
            print(f"Конфигурация: '{best_config_name}'")
            print(f"  Best Val Levenshtein: {best_run['best_val_lev']:.4f} (Эпоха {best_run.get('best_epoch', 'N/A')})")

            # Сравнение с предыдущим лучшим результатом (0.726)
            previous_best = 0.726
            print(f"\nСравнение с предыдущим лучшим (Single Classifier, ~{previous_best:.4f}):")
            if best_run['best_val_lev'] < previous_best:
                print(f"  >>> УЛУЧШЕНИЕ! Double Classifier для модели 3x512 работает лучше. <<<")
                print("  >>> РЕКОМЕНДАЦИЯ: Использовать эту конфигурацию для финального обучения. <<<")
            elif abs(best_run['best_val_lev'] - previous_best) < 0.005: # Если разница очень мала
                 print(f"  >> Результат очень близок к Single Classifier. Можно выбрать любой вариант или оставить Single для простоты. <<")
            else:
                print(f"  >> Ухудшение. Single Classifier для модели 3x512 остается лучшим вариантом. <<")
                print("  >>> РЕКОМЕНДАЦИЯ: Использовать конфигурацию с Single Classifier для финального обучения. <<<")

        else: print("\nНе удалось найти валидный результат.")
    else: print("\nНе удалось определить результат.")

    results_csv_path = OUTPUT_DIR / f"final_clsdouble_test_results.csv"
    try: results_df.to_csv(results_csv_path, index=False); print(f"\nПолные результаты сохранены в CSV: {results_csv_path}")
    except Exception as e: print(f"\nНе удалось сохранить результаты в CSV: {e}")
# ============================================================================
# === КОНЕЦ БЛОКА 3 ===
# ============================================================================

print(f"\n--- Ячейка 13: Завершена (Тест Double Classifier) ---")
print("-" * 50)
# =============================================================================

--- Ячейка 13: Запуск Цикла Обучения (Тест Double Classifier для лучшей модели 3x512) ---

1. Определение конфигурации для теста Double Classifier...

Конфигурация для запуска (1):
  - BestModel_3x512_ClsDouble: RNN=3x512, Drop=0.2, Cls=double, Trim OFF

2. Копирование базовой конфигурации обучения...

--- Начало цикла запусков (1 итерация) ---
MLflow эксперимент: 'Morse_Final_ClsDouble_Test'

--- Попытка завершить ЛЮБОЙ активный MLflow run перед циклом ---
--- Активный MLflow run не обнаружен. ---

========== Запуск 1/1: Конфигурация 'BestModel_3x512_ClsDouble' ==========
  Аудио параметры: {"sample_rate": 8000, "frame_length_rms": 128, "hop_length_rms": 64, "apply_filter": false, "apply_trimming": false, "trim_top_db": 30}
  Параметры модели: {"input_feature_dim": 2, "cnn_out_channels": [64, 128, 128], "cnn_kernel_size": 9, "cnn_stride": 1, "cnn_padding": "same", "cnn_pool_kernel": 2, "rnn_hidden_size": 512, "rnn_num_layers": 3, "dropout_rate": 0.2, "activation_fn": "GELU", "classifi

Эпоха 1/15 [Тренировка]:   0%|                                                                                …

   [Валидация]:   0%|                                                                                         …

  Итоги Эпохи 1: Train Loss=3.7650, Train Lev=8.1727, Val Loss=1.0587, Val Lev=2.2693
  ✨ Val Lev улучшился: inf -> 2.2693. Сохранение...

--- Эпоха 2/15 ---


Эпоха 2/15 [Тренировка]:   0%|                                                                                …

...

  Итоги Эпохи 15: Train Loss=0.1463, Train Lev=0.4179, Val Loss=0.3094, Val Lev=0.6817
  Val Lev не улучшился (0.6817 vs best 0.6790). Без улучшений: 1/7

  Обучение завершено (completed) за 189.78 мин. Лучший Val Lev: 0.6790 (Эпоха 14)

4. Сохранение артефактов...
  Лучшая модель сохранена: model_SR8k_F128h64_FiltOFF_CNN128_RNN3x512_GELU_Clssingle_LR2e-04_WD1e-04_BestModel_3x512_ClsDouble.pth
  Параметры сохранены: params_SR8k_F128h64_FiltOFF_CNN128_RNN3x512_GELU_Clssingle_LR2e-04_WD1e-04_BestModel_3x512_ClsDouble.json

5. Запуск инференса...
  Загрузка: model_SR8k_F128h64_FiltOFF_CNN128_RNN3x512_GELU_Clssingle_LR2e-04_WD1e-04_BestModel_3x512_ClsDouble.pth, params_SR8k_F128h64_FiltOFF_CNN128_RNN3x512_GELU_Clssingle_LR2e-04_WD1e-04_BestModel_3x512_ClsDouble.json
Архитектура MorseRecognizer:
  CNN: 3 layers, OutChannels=[64, 128, 128], Kernel=9, Pool=2
       Output Dim=128, Time Reduction Factor=8.0x
  RNN: BiGRU, Layers=3, Hidden Size=512
       Output Dim=1024
  Activation: GELU
  Cl

Инференс:   0%|                                                                                               …

  Инференс завершен за 96.36 сек. Предсказаний: 5000/5000
  Формирование submission: submission_greedy_SR8k_F128h64_FiltOFF_CNN128_RNN3x512_GELU_Clssingle_LR2e-04_WD1e-04_BestModel_3x512_ClsDouble.csv
  Submission сохранен.
==================== Завершение: _BestModel_3x512_ClsDouble ====================

--- Успешное завершение run_training_pipeline для 'BestModel_3x512_ClsDouble' ---
--- Завершение обработки Запуска 1/1 ('BestModel_3x512_ClsDouble') ---

--- Цикл запусков завершен за 3.19 часов ---

--- Итоги Теста Double Classifier ---

Сводная таблица результатов:


,config_name,trimming_status,best_val_lev,best_epoch,status,train_time_min,error
0,BestModel_3x512_ClsDouble,OFF,0.6790,14,completed,189.7841,None



--- Результат Теста Double Classifier ---
Конфигурация: 'BestModel_3x512_ClsDouble'
  Best Val Levenshtein: 0.6790 (Эпоха 14)

Сравнение с предыдущим лучшим (Single Classifier, ~0.7260):
  >>> УЛУЧШЕНИЕ! Double Classifier для модели 3x512 работает лучше. <<<
  >>> РЕКОМЕНДАЦИЯ: Использовать эту конфигурацию для финального обучения. <<<

Полные результаты сохранены в CSV: C:\Users\vasja\OneDrive\Рабочий стол\MorseAudioDecoder\output_final_trim_test\final_clsdouble_test_results.csv

--- Ячейка 13: Завершена (Тест Double Classifier) ---
--------------------------------------------------
